# Projeto 2 - Ciência dos Dados

Nome: Amanda Rosa do Carmo

Nome: Beatriz Cabral

Nome: Sophia Kerber Shigueoka

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [3]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [144]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

In [145]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[!\-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

___
## Autenticando no  Twitter

* Conta: ***@KerberSophie***

In [4]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @KerberSophie

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [6]:
#Produto escolhido:
produto = '13 Reasons Why'

#Quantidade mínima de mensagens capturadas:
#Escolhemos 1480 a partir de um método empírico. Quando testamos com 1000 não conseguimos tweets suficientes, e chegamos
#a conclusão de que o problema era proveniente do set(), que diminuía a quantidade de tweets baixados. Assim, decidiu-se
#aumentar a quantidade de tweets para 1500, já que com 1000 ficaram falatando 360 tweets. Com 1500, ficamos com 20 tweets a mais.
#como eram somente 20 tweets, imaginamos que poucos deles seriam retweets, então tiramos exatamente 20 tweets dos 1500.
#Com 1480 tweets, conseguimos 399 tweets de teste, ou seja, faltou somente 1. não achamos que 1 tweets afetará tanto a base
# de dados, então decidimos manter os 1480.
n = 1480
#Quantidade mínima de mensagens para a base de treinamento:
t = 600

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [7]:
#Cria um objeto para a captura
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
list = []
shuffle(msgs)

#Usando o set() para tirar os retweets
listaset = set(msgs)

#Adicionando o resultado do set() a uma lista para evitar o erro TypeError: 'set' object is not subscriptable
for value in listaset:
    list.append(value)

TweepError: Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/search/tweets.json?q=13+Reasons+Why&lang=pt (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')")))

Salvando os dados em uma planilha Excel:

In [41]:
#Linhas abaixo foram comentadas uma vez que o conjunto de dados era o mesmo, mas o número de tweets foi alterado.

# #Verifica se o arquivo não existe para não substituir um conjunto pronto
# if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
writer = pd.ExcelWriter('{0} novo.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
dft = pd.DataFrame({'Treinamento' : pd.Series(list[:t])})
dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

dfc = pd.DataFrame({'Teste' : pd.Series(list[t:])})
dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [146]:
excel = pd.read_excel('13 Reasons Why v2.xlsx', sheet_name='Treinamento')
excel.head()

,Treinamento,Categoria
0,alguém aq ja assistiu a 3° temp de 13 reasons ...,0
1,(na 3° temporada de 13 reasons why) eu estou t...,R
2,13 ep de 13 reasons why p tentar fazer vc sent...,N
3,nossa nem sabia que tinha lançado a 3° tempora...,0
4,estava começando a ver o primeiro episódio de...,N


In [147]:
excel_treino = pd.read_excel('13 Reasons Why v2.xlsx', sheet_name='Teste')
excel_treino.rename(columns={'Unnamed: 1':'Clas_manual'}, inplace=True)
excel_treino.head()

,Teste,Clas_manual
0,a coisa mais idiota de 13 reasons why foi colo...,N
1,atribuí nota 8 ao episódio 3x1 - yeah. i'm the...,0
2,rt @citou13reasons: ''quando você julga uma pe...,0
3,"terminei 13 reasons why também, meu hobbie é c...",R
4,rt @wivisampaio: saiu a nova temporada de elit...,0


___
### Função para limpar e criar tabela de frequências relativas:

In [157]:
def FreqRel(cat):
    texto_completo = ' '.join(excel[excel.Categoria==cat].Treinamento)
    semhttp = re.sub(r'http\S+', '', texto_completo)
    text_limpo = cleanup(semhttp)
    rel = text_limpo.split()
    freq_rel = pd.Series(rel).value_counts(True)
    return freq_rel

In [158]:
# # Estas funções dividem a fun
# def Limpa(texto):
#     semhttp = re.sub(r'http\S+', '', texto)
#     text_limpo = cleanup(semhttp)
#     rel = text_limpo.split()
#     return(rel)

# def FreqRel(Limpa):
#     freq_rel = pd.Series(Limpa).value_counts(True)
#     return freq_rel

___
### Descobrindo P(N), P(R), P(P) e P(0)

In [166]:
texto_1 = ' '.join(excel[excel.Categoria==0].Treinamento)
texto_2 = ' '.join(excel[excel.Categoria=='R'].Treinamento)
texto_3 = ' '.join(excel[excel.Categoria=='P'].Treinamento)
texto_4 = ' '.join(excel[excel.Categoria=='N'].Treinamento)

texto_1 = Limpa(texto_1)
texto_2 = Limpa(texto_2)
texto_3 = Limpa(texto_3)
texto_4 = Limpa(texto_4)

#juntando todos os textos
texto_enorme = texto_1 + texto_2 + texto_3 + texto_4
set(texto_enorme)
dic_global = 0
for palavra in texto_enorme:
        dic_global+=1
    
print('Todas as palavras somadas:')
print(dic_global)

def prob_cat(cat):
    palavras = 0
    texto_completo = ' '.join(excel[excel.Categoria==cat].Treinamento)
    texto_completo.replace('•', '')
    semhttp = re.sub(r'http\S+', '', texto_completo)
    text_limpo = cleanup(semhttp)
    rel = text_limpo.split()
    set(rel)
    # Para verificar se o código era coerente, imprimimos o total de palavras de cada categoria
    for palavra in rel:
        palavras += 1
    prob_final = palavras/dic_global
    return prob_final

Todas as palavras somadas:
10097


In [167]:
print("Probabilidade da Categoria 'Neutro'")
print(prob_cat(0))
print('')
print("Probabilidade da Categoria 'Negativo'")
print(prob_cat('N'))
print('')
print("Probabilidade da Categoria 'Positivo'")
print(prob_cat('P'))
print('')
print("Probabilidade da Categoria 'Reação'")
print(prob_cat('R'))



print('')
print("Probabilidade Total (se tudo der certo, deve ser 1)")
print(prob_cat(0) + prob_cat('N') + prob_cat('P') + prob_cat('R'))

Probabilidade da Categoria 'Neutro'
0.5090620976527681

Probabilidade da Categoria 'Negativo'
0.22194711300386252

Probabilidade da Categoria 'Positivo'
0.11379617708230168

Probabilidade da Categoria 'Reação'
0.15519461226106765

Probabilidade Total (se tudo der certo, deve ser 1)
1.0


In [178]:
FreqRel('R')['things']

KeyError: 'things'

In [153]:
Neutro_set = set(FreqRel(0).index)
Reacao_set = set(FreqRel('R').index)
inter = Neutro_set.intersection(Reacao_set)

___
### Additive Smoothing


1. Estimando o tamanho do vocabulário desconhecido: 10ˆ6
2. Cada palavra ganha um bônus alpha, de modo de alpha = 1
3. A frequência normalizada: f = (wi + alpha)/soma(wj + alpha) + alpha*v -- para wi conhecido

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

P(N|palavras) = P(N) * p(cada_palavra|N)/p()
P(P|palavras) = 
P(0|palavras) = 
P(R|palavras) = 

In [154]:
# dic_prob = {}
# for palavra in tweet:
#     #key: palavra 
#     #value: P(palavra|categoria)
#     dic_prob[palavra] =  P(palavra)/prob_cat()

In [155]:
def probabilidade():
    for tweet in excel_treino['Teste']:
        tweet = cleanup(linha.lower())
        pal_tweet = tweet.split()
        for palavra in pal_tweet:
            probR = (prob_cat('R') * (FreqRel('R')[palavra]))
            probP = (prob_cat('P') * (FreqRel('P')[palavra]))
            probN = (prob_cat('N') * (FreqRel('N')[palavra]))
            prob0 = (prob_cat('0') * (FreqRel('0')[palavra]))
            dic_cat = {}
            dic_cat[R] = probR
            dic_cat[P] = probP
            dic_cat[N] = probN
            dic_cat[0] = prob0
        #Não precisa dividir pela probabilidade do tweet pois cancelaria.
    return (max(dic_cat.items(), key=operator.itemgetter(1))[0])

Probabilidade()
        #prob = prob_cat(categoria) * prob_

    
    
# fazer contador pra ver qual é a maior probabilidade, e adicionar em uma nova coluna do dataframe
#  DataFrame.insert(2, 'Predição', [lista com letra de cada linha], True)



KeyError: '•'

In [156]:
for palavra in tweet_teste_neutro:
    probabilidade_neutro = 

SyntaxError: invalid syntax (<ipython-input-156-109dc68a5cdf>, line 2)

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**